In [1]:
# Create the Solver object for either NEOS or a local solver
def SetUpSolver(Model):
    Model.Options = None
    if Neos:
        Solver = pyo.SolverManagerFactory('neos')   # Solver on NEOS
        if pyo.value(Model.Engine) == 'cplex':   # Linear
            Model.Options = {'timelimit': Model.TimeLimit}
        elif pyo.value(Model.Engine) == 'octeract':   # Linear or non-linear
            Model.Options = {'MAX_SOLVER_TIME': Model.TimeLimit, 'MILP_SOLVER': 'HIGHS'}
        elif pyo.value(Model.Engine) == 'couenne':   # Non-linear
            print('No options for Couenne')
        else:
            print('Unknown NEOS solver when setting options')
    else:
        Solver = pyo.SolverFactory(pyo.value(Model.Engine))   # Local solver installed
        if pyo.value(Model.Engine) == 'couenne':   # Non-linear
            print('No options for Couenne') # Couenne doesn't accept command line options, use couenne.opt instead
        elif pyo.value(Model.Engine) == 'appsi_highs':   # Linear
            Solver.options['time_limit'] = pyo.value(Model.TimeLimit)
        else:
            print('Unknown local solver when setting options')
    
    return Solver, Model

In [2]:
# Call either NEOS or a local solver
def CallSolver(Solver, Model):
    if Neos:
        if Model.Options == None:
            Results = Solver.solve(Model, load_solutions = LoadSolution, tee = Verbose, solver = Model.Engine)
        else:
            Results = Solver.solve(Model, load_solutions = LoadSolution, tee = Verbose, solver = Model.Engine, options = Model.Options)
    else:
        Results = Solver.solve(Model, load_solutions = LoadSolution, tee = Verbose)
    
    return Results, Model